- DB서버의 정보
    - 주소
    - port
    - user명
    - 비밀번호
    - 데이터베이스명
- 서버의 정보들이 외부에 노출되면 보안 문제가 발생할 수 있다.
    - 서버의 정보와 같이 외부에 노출되는 데이터는 dotenv(.env)를 이용하여 데이터를 숨겨서 사용한다.(환경 변수 사용)

In [162]:
# dotenv 라이브러리 설치
# !pip install python-dotenv

In [236]:
import pandas as pd
import pymysql
import os     # env라는 환경변수가 만들어져 있는 파일에 접근하기 위해
from dotenv import load_dotenv

In [237]:
# 파일명 '.env'으로 새 파일 만들어서 서버 정보 작성 후 저장
# .env 파일을 환경 변수에 등록
load_dotenv()  # 아무것도 안 적으면 현재 위치가 기본값
# True가 출력되면 잘 불러와진 것.

True

In [238]:
# 등록된 환경 변수에 접근
# 획득하다(get) + 환경 변수(env) -> getenv
# getenv( {변수명} )
os.getenv( 'db_name' )

'multicam'

In [239]:
# getenv는 return data의 type이 무조건 str이므로, 숫자가 필요하면 변환해줘야 한다.
os.getenv( 'pw' )

'1408'

In [240]:
# DB 서버와의 연결
# mac 사용자는 터미널에서 brew services start mysql
# window는 백그라운드에서 기본적으로 돌아가고 있음
_db = pymysql.connect(
    host= os.getenv('host'),
    port= int(os.getenv('port')),
    user= os.getenv('user'),
    password= os.getenv('pw'),
    db= os.getenv('db_name')
)

In [241]:
# 가상 공간 cursor 생성 (연결된 데이터베이스로부터 생성)
cursor = _db.cursor( pymysql.cursors.DictCursor )
    # = pymysql 라이브러리 안의 cursor들 중, dict 형태로 받기

In [242]:
create_user_query = """
    CREATE TABLE if not exists `user_info`
    (
        `id` varchar(35) primary key,
        `password` varchar(45) not null,
        `name` varchar(45),
        `birth` varchar(10)
    )
"""

cursor.execute(create_user_query)

0

In [245]:
# query문 안에 유저가 입력한 데이터를 대입하여 query 실행
# 회원 가입
# - 입력 데이터가 4개 (id, password, name, birth)
input_id= input('ID를 입력하시오.')
input_password = input('PassWord를 입력하시오.')
input_name = input('Name을 입력하시오.')
input_birth = input('Birth를 입력하시오.')

# 회원 정보를 저장하는 query 생성
signup_query = '''
    insert into
    `user_info`
    values
    ( %s, %s, %s, %s )
'''

# signup_query와 데이터를 이용하여 execute() 사용
cursor.execute( signup_query, [input_id, input_password, input_name, input_birth] )
    # ( 쿼리문(str), 해당하는 쿼리문에 들어갈 args(agrements, 요소들) )

IntegrityError: (1062, "Duplicate entry 'test1234' for key 'user_info.PRIMARY'")

---

CRUD 중
- CUD (insert, update, delete)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성
    - execute(  ) 함수를 이용해 cursor에 질의 보내기
        - cursor에 존재하는 table의 data 변화
    - DB server와 cursor 동기화 (  .commit(  )  )
    - DB server와의 연결 종료

- R (select)
    - query 작성
        - 데이터가 존재한다면 데이터들을 리스트나 튜플의 형태로 생성
    - execute(  ) 함수를 이용해 cursor에 질의 보내기
        - cursor에 존재하는 table의 data 변화 X
    - execute(  ) 함수의 결과를 불러와 변수에 저장 (  fetchall(  )  )
    - DB server와의 연결 종료

In [246]:
# query문이 select문인가?
# select를 소문자로 써도 되지만, 일반적으로는 대문자로 쓴다.
# 쿼리로 만들어진 텍스트는 대/소문자를 구분하지 않아, 첫 번째에 있는 명령어가 중요하다.
test_query = """
    SELECT
    *
    from
    `table`
"""
test_query

'\n    SELECT\n    *\n    from\n    `table`\n'

In [247]:
# query문이 select문인가?
# 좌우의 공백 제거( strip() )
# -> 위치를 이용해 문자 나눠주기
# -> 문자열을 대/소문자로 통일
# -> 비교
text_data = test_query.strip().lower()[ :6 ]
text_data

'select'

In [248]:
# 문자를 소문자로 변환 ( lower() )
# -> 공백을 기준으로 데이터를 나눠주기 ( split() )
# -> 첫 번째 원소[0]를 선택
# -> 비교
test_query.lower().split()[0] == text_data
# '\n'은 문자가 아닌 공백으로 친다.

True

In [249]:
# 문자를 소문자로 변환 ( lower() )
# -> 좌우의 공백 제거 ( strip() )
# -> 시작하는 문자가 text_data와 같은지 확인하는 함수 호출 ( startswith() )
test_query.lower().strip().startswith(text_data)

True

---

In [250]:
# CRUD query를 모두 사용할 수 있는 함수 생성
# 매개변수 2개 - query문, 그에 대입되는 데이터들(가변)
def sql_query(
        _query, *_data_list
):
    # _query는 sql query문이 문자로 입력되는 부분
    # _query가 select문인가?
    if _query.lower().strip().startswith('select'):
        # cursor에 질의 보내기
        cursor.execute(_query, _data_list)
        # select문에 대한 결과 불러오기
        result = cursor.fetchall()
    else:
        # cursor에 질의 보내기
        cursor.execute(_query, _data_list)
        result = 'Query OK'

In [251]:
# 위처럼 해도 되지만, if-else문 안에 겹치는 코드(cursor.execute(_query, _data_list))가 있다.
# Error 발생에 대한 대처를 위해 try-except 구문도 넣어준다.

# CRUD query를 모두 사용할 수 있는 함수 생성
# 매개변수 2개 - query문, 그에 대입되는 데이터들(가변)
def sql_query(
        _query, *_data_list
):
    try:
        # cursor에 질의 보내기
        cursor.execute(_query, _data_list)
    # excute() 함수에 Error가 발생한다면?
    except Exception as e:
        print(e)
        # 함수를 강제로 종료하며 메시지 되돌려주기
        return 'Query Error'
    
    # _query는 sql query문이 문자로 입력되는 부분
    # _query가 select문인가?
    if _query.lower().strip().startswith('select'):
        # select문에 대한 결과 불러오기
        result = cursor.fetchall()
    else:
        result = 'Query OK'
    return result

In [252]:
test_query = """
    select
    *
    from
    `user_info`    
"""
sql_query(test_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1111', 'password': '1111', 'name': 'hyun', 'birth': '011111'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'}]

In [253]:
# 로그인 과정
# 유저가 입력한 id, password를 이용해서 데이터베이스에 데이터가 존재하는가?
input_id = input('id 입력하시오')
input_password = input('PassWord를 입력하시오.')

login_query = '''
    select * from
    `user_info`
    where
    `id` = %s
    and
    `password` = %s
'''

login_result = sql_query(login_query, input_id, input_password)  # 매개변수(*_data_list)가 가변이므로 input_id, input_password를 list로 묶어줄 필요 X

In [254]:
# id나 password를 틀리면 빈 값이 출력됨
login_result

[{'id': '', 'password': '', 'name': '', 'birth': ''}]

In [255]:
if login_result:
    # = login_result에 데이터가 존재한다면
    # = 길이가 0이 아니면
    print('로그인 성공')
else:
    # = login_result에 데이터가 존재하지 않는다면
    # = 길이가 0이면
    print('로그인 실패')

로그인 성공


---

In [256]:
# insert 테스트

input_id = 'test0000'
input_password = '1234'
input_name = 'park'
input_birth= '000101'

insert_query = """
    insert into
    `user_info`
    values
    (%s, %s, %s, %s)
"""

sql_query(insert_query, input_id, input_password, input_name, input_birth)

(1062, "Duplicate entry 'test0000' for key 'user_info.PRIMARY'")


'Query Error'

In [257]:
# 데이터베이스 서버와 cursor를 동기화하고 서버와의 연결을 종료하는 함수 생성
def commit_db():
    # cursor와 데이터베이스 서버와의 동기화
    _db.commit()
    print('Commit 완료')       # 중간 상태메시지
    # 데이터베이스 서버와의 연결을 종료
    _db.close()
    print('서버와의 연결 종료')  # 중간 상태메시지
# return 필요 없음

In [258]:
commit_db()

# 메시지 뜨면 MySQL Workbench 실행해
# multicam > Tables > user_info의 Result Grid에 지금까지 입력한 데이터가 잘 연동되었는지 확인

Commit 완료
서버와의 연결 종료


---
---

In [259]:
# 데이터베이스 서버와 연결하여 query문을 사용하고 서버와의 연결을 종료하는 class 생성

# class 선언
class MyDB:
    # 생성자 함수
    # = class 내부에서 사용하려는 변수의 데이터를 대입하는 함수로,
    # class가 생성될 때 한 번만 실행되는 함수
    # 입력받을 데이터: 데이터베이스 서버의 정보 -> 기본값 설정(로컬 PC의 데이터베이스 정보)
    def __init__(
            self,
            _host = '127.0.0.1',
            _port = 3306,
            _user = 'root',
            _pw = '1408',
            _db_name = 'multicam'
    ):
        # self.변수 생성
        # class에서 사용할 서버의 정보를 변수에 저장

        self.host = _host
        self.port = _port
        self.user = _user
        self.pw = _pw
        self.db_name = _db_name

    # query문과 데이터를 입력 받아서 데이터베이스 서버에 질의를 보내는 함수
    def sql_query(
            self,
            _query,
            *_data_list
    ):
        # _query 매개변수는 기본값이 존재하지 않으므로 필수 입력 공간
        # _data_list는 인자의 개수를 가변으로 받는다. 개수가 0개면 ()을 생성

        # 데이터베이스 서버와의 연결( _db 변수를 생성하여 연결 )
        # pymysql.connect() 함수는 입력값이 서버의 정보이다.
            # 서버의 정보는 self.host, self.port, self.user, self.pw, self.db_name에 저장되어 있다.
        # connect() 함수에 해당 데이터를 입력
        try:
            # self._db 라는 변수가 존재하는지 확인
            self._db
            # 해당 변수가 선언되어있지 않다면 NameError 발생
        except:
            # self.__db라는 변수가 존재하지 않을 때
            self._db = pymysql.connect(
            host= self.host,
            port= self.port,
            user= self.user,
            password= self.pw,
            db= self.db_name
        )   # self 쓰는 이유: 연결하는 데이터베이스가 여러 개일 수 있으므로 자기자신 안에 공간을 만들어 연결할 데이터를 넣어둔다.
    
        # self._db를 이용하여 cursor 생성
        # ( class 내부의 다른 함수에서도 사용 가능하도록 self.변수로 등록)
        self.cursor = self._db.cursor( pymysql.cursors.DictCursor )

        # _query와 _data_list를 이용하여 self.cursor에 질의를 보낸다.
        # 그 과정에서 문제가 발생하면 예외 처리를 한다. (try-except)
        try:
            self.cursor.execute( _query, _data_list )
        except Exception as e:
            print(e)
            # query문에서 문제가 발생했으니 Error 메시지를 돌려주고 함수 종료
            return 'Query Error'
        
        # _query가 select문이라면?
        if _query.lower().strip().startswith('select'):
            # query문이 select문인 경우
            # self.cursor에서 결과물을 돌려받는다.
            result = self.cursor.fetchall()
        else:
            # select문이 아닌 경우
            result = 'Query OK'
        return result

    def db_commit(self):
        try:
            # self._db와 self.cursor 동기화
            self._db.commit()
            print('commit 완료')
            # 데이터베이스 서버와의 연결 종료
            self._db.close()
            print('서버와의 연결 종료')
            # 변수를 삭제
            del self._db
        except Exception as e:
            print(e)
            # self._db와 self.cursor가 생성되지 않은 상황
            # (서버와 연결되지 않은 상태에서 실행 시 Error)
            print('''데이터베이스 서버와의 연결이 없습니다. 
                sql_query() 함수를 이용하여 서버와 연결해주세요.''')

In [ ]:
# MyDB class 생성하는데 생성자 함수의 매개변수는 기본값으로 사용
# = 로컬 PC에 있는 데이터베이스에 연결
    # MyDB 뒤에 () 자꾸 빼먹음!!!주의!!!! -> TypeError: MyDB.sql_query() missing 1 required positional argument: '_query'
db1 = MyDB()

In [261]:
# 외부의 데이터베이스에 연결
db2 = MyDB(
    '192.168.50.40',
    3306,
    'test',
    '1234',
    'multicam'
)

In [262]:
select_query = '''
    select
    *
    from
    `user_info`
'''

In [263]:
select_query

'\n    select\n    *\n    from\n    `user_info`\n'

In [264]:
db1.sql_query(select_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '1234', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1111', 'password': '1111', 'name': 'hyun', 'birth': '011111'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'}]

---
### update문 - 수정

In [265]:
# %s만 쓸 때는 따옴표로 감싸면 안 됨 (그 자체로 문자 취급이므로)

update_query = '''
    update
    `user_info`
    set
    `password` = %s
    where
    `id` = %s
'''

In [266]:
db1.sql_query(update_query, '0123', 'test')

'Query OK'

In [267]:
db1.sql_query(select_query)

[{'id': '', 'password': '', 'name': '', 'birth': ''},
 {'id': 'test', 'password': '0123', 'name': 'kim', 'birth': '900101'},
 {'id': 'test0000', 'password': '1234', 'name': 'park', 'birth': '000101'},
 {'id': 'test1111', 'password': '1111', 'name': 'hyun', 'birth': '011111'},
 {'id': 'test1234', 'password': '0000', 'name': 'kim', 'birth': '900101'}]

---

In [268]:
# database.py 생성하고 만들어둔 MyDB 클래스 복붙

In [269]:
import database

In [280]:
# database 모듈 안에 있는 MyDB 생성
mod_db = database.MyDB()

In [281]:
# mod_db에 입력되는 데이터베이스 서버의 정보는? = 로컬
select_query2 = '''
    select * from `user_info`
'''

db_data = mod_db.sql_query(select_query2)

In [277]:
pd.DataFrame(db_data)

,id,password,name,birth
0,,,,
1,test,1234,kim,900101
2,test0000,1234,park,000101
3,test1111,1111,hyun,011111
4,test1234,0000,kim,900101


In [273]:
# database.py 파일을 python 환경변수에 이동

# 환경 변수 목록 확인을 위해 라이브러리 호출
import sys

In [ ]:
sys.path[5].replace('\\', '/')
# 출력된 경로에 database.py 파일을 복붙하면 어디서든 사용 가능해짐

'C:/Users/student/AppData/Roaming/Python/Python313/site-packages'